In [1]:
!pip install pandas openpyxl xlrd

Defaulting to user installation because normal site-packages is not writeable


In [26]:
import pandas as pd

# --- Step 1: Read Excel ---
file_path = r"E:\OrderProcssing\data\NGL_PO.xlsx"
df = pd.read_excel(file_path , skiprows=1)
rename_map = {
    'Unnamed: 0': 'SrNo',
    'Unnamed: 1': 'ItemSize',
    'Unnamed: 2': 'StyleCode',
    'Unnamed: 3': 'Metal',
    'Unnamed: 4': 'Gold Rate',
    'Unnamed: 5': 'Gold Wt',
    'Unnamed: 6': 'Stone Wt',
    'Unnamed: 7': 'Metal Wt',
    'Unnamed: 8': 'Dia Pcs',
    'Set Rate': 'SetRate',
    'Set Val': 'SetValue',
    'Unnamed: 19': 'LabourRate',
    'Total Labour': 'TotalLabour',
    'Unnamed: 21': 'OtherCharges',
    'Unnamed: 22': 'MakingCharges',
    'Unnamed: 23': 'Wastage',
    'Unnamed: 24': 'FinalValue',
    'Unnamed: 25': 'Discount',
    'Unnamed: 26': 'NetValue'
}

df.rename(columns=rename_map, inplace=True)

# # --- Step 2: Keep only rows where 'Sr No.' is not NaN ---
df_clean = df[df['SrNo'].notna()].reset_index(drop=True)
#df_clean
selected_columns = [
    'SrNo', 'StyleCode','ItemSize', 'Metal','Dia Qlty'
]

df_selected = df[selected_columns].copy()

order_qty_input = input("Enter OrderQty value to assign for all rows: ")
item_po_input = input("Enter ItemPoNo value for all rows: ")
priority_input = input("Enter Priority value for all rows: ")
additional_input = input("Enter additional string to append after Dia Qlty (e.g. ' (W7A),Emerald ROUND (AA)'): ")

# --- Step 4: Add "OrderQty" and "OrderItemPcs" after "ItemSize" ---
itemsize_pos = df_selected.columns.get_loc('ItemSize') + 1

df_selected.insert(itemsize_pos, 'OrderQty', order_qty_input)
df_selected.insert(itemsize_pos + 1, 'OrderItemPcs', '')  

metal_mapping = {
    '585/W': 'G585W',
    '585/Y': 'G585Y',
    '585/R': 'G585P',
    '585/P': 'G585P',# Assuming R = Pink/Red gold
    '585/RW': 'G585PW',
    '585/PW': 'G585PW',# If combination exists
    '750/W': 'G750W',
    '750/Y': 'G750Y',
    '750/R': 'G750P',
    '750/P': 'G750P',
    '750/RW': 'G750PW',
    '750/PW': 'G750PW'
}

# Strip any extra spaces before mapping
df_selected['Metal'] = df_selected['Metal'].str.strip().map(metal_mapping).fillna('')

# --- Step 6 (Optional): Add 'Tone' column based on last letter of Metal code ---
df_selected['Tone'] = df_selected['Metal'].str.extract(r'(W|Y|P)$')[0].fillna('')

# --- Step 7: Move 'Tone' column right after 'Metal' ---
tone_col = df_selected.pop('Tone')
df_selected.insert(df_selected.columns.get_loc('Metal') + 1, 'Tone', tone_col)

# --- Step 9: Add the new columns after 'Tone' ---
tone_pos = df_selected.columns.get_loc('Tone') + 1

new_cols = ['ItemPoNo', 'ItemRefNo', 'StockType', 'Priority', 'MakeType','CustomerProductionInstruction']

for col in new_cols:
    if col == 'ItemPoNo':
        df_selected.insert(tone_pos, col, item_po_input)
    elif col == 'Priority':
        df_selected.insert(tone_pos, col, priority_input)
    else:
        df_selected.insert(tone_pos, col, '')
    tone_pos += 1  # increment position for next column

cpi_pos = df_selected.columns.get_loc('CustomerProductionInstruction') + 1

def create_special_remarks(row):
    dia = row['Dia Qlty'] if pd.notna(row['Dia Qlty']) else ''
    return f"DIA QLTY-{dia}{additional_input}"

df_selected.insert(cpi_pos, 'SpecialRemarks', df_selected.apply(create_special_remarks, axis=1))

dpi_pos = df_selected.columns.get_loc('SpecialRemarks') + 1

def map_design_instruction(tone):
    if pd.isna(tone):
        return ''
    tone = tone.strip().upper()
    if tone == 'W':
        return 'WHITE RODIUM'
    elif tone in ['Y', 'PW', 'PY']:
        return 'NO RODIUM'
    else:
        return ''

df_selected.insert(dpi_pos, 'DesignProductionInstruction', df_selected['Tone'].apply(map_design_instruction))

stamp_pos = df_selected.columns.get_loc('DesignProductionInstruction') + 1

df_selected.insert(
    stamp_pos,
    'StampInstruction',
    df_selected['Metal'].apply(lambda x: f"{x}+ LOGO" if pd.notna(x) and x != '' else '')
)

stamp_pos = df_selected.columns.get_loc('StampInstruction') + 1

new_columns = [
    'OrderGroup', 'Certificate', 'SKUNo',
    'Basestoneminwt', 'Basestonemaxwt',
    'Basemetalminwt', 'Basemetalmaxwt',
    'Productiondeliverydate', 'Expecteddeliverydate'
]

for col_name in new_columns:
    df_selected.insert(stamp_pos, col_name, '')
    stamp_pos += 1  # increment position for the next column

df_selected.drop(columns=['Dia Qlty'], inplace=True) 
df_selected.to_csv(r"C:\Users\Admin\Desktop\NGL_PO_Cleaned.csv", index=False)
    
df_selected.head(1)


Enter OrderQty value to assign for all rows:  1
Enter ItemPoNo value for all rows:  Saumil Email 03-31-25
Enter Priority value for all rows:  REG
Enter additional string to append after Dia Qlty (e.g. ' (W7A),Emerald ROUND (AA)'):  (W7A),Emerald ROUND (AA)


,SrNo,StyleCode,ItemSize,OrderQty,OrderItemPcs,Metal,Tone,ItemPoNo,ItemRefNo,StockType,...,StampInstruction,OrderGroup,Certificate,SKUNo,Basestoneminwt,Basestonemaxwt,Basemetalminwt,Basemetalmaxwt,Productiondeliverydate,Expecteddeliverydate
0,1.0,BR0126CF,7. INCH,1,,G585W,W,Saumil Email 03-31-25,,,...,G585W+ LOGO,,,,,,,,,
